<a href="https://colab.research.google.com/github/littlemisslilycane/PS2/blob/master/ner_decoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementing a Viterbi Decoder and Evaluation for Sequence Labeling

In this assignment, you will build a Viterbi decoder for an LSTM named-entity recognition model. As we mentioned in class, recurrent and bidirectional recurrent neural networks, of which LSTMs are the most common examples, can be used to perform sequence labeling. Although these models encode information from the surrounding words in order to make predictions, there are no "hard" constraints on what tags can appear where.

There hard constraints are particularly important for tasks that label spans of more than one token. The most common example of a span-labeling task is named-entity recognition (NER). As described in Eisenstein, Jurafksy & Martin, and other texts, the goal of NER is to label spans of one or more words as _mentions_ of an _entity_, such as a person, location, organization, etc.

The most common approach to NER is to reduce it to a sequence-labeling task, where each token in the input is labeled either with an `O`, if it is "outside" any named-entity span, or with `B-TYPE`, if it is the first token in an entity of type `TYPE`, or with `I-TYPE`, if it is the second or later token in an entity of type `TYPE`. Distinguishing between the first and later tokens of an entity allow us to identify distinct entity spans even when they are adjacent.

Common values of `TYPE` include `PER` for person, `LOC` for location, `DATE` for date, and so on. In the dataset we load below, there are 17 distinct types.

The span-labeling scheme just described implies that the labels on tokens must obey certain constraints: the tag `I-PER` must follow either `B-PER` or another `I-PER`. It cannot follow `O`, `B-LOC`, or `I-LOC`, i.e., a tag for a different entity type. By themselves, LSTMs or bidirectional LSTMs cannot directly enforce these constraints. This is one reason why conditional random fields (CRFs), which _can_ enforce these constraints, are often layered on top of these recurrent models.

In this assignment, you will implement the simplest possible CRF: a CRF so simple that it does not require any training. Rather, it will assign weight 1 to any sequence of tags that obeys the constraints and weight 0 to any sequence of tags that violates them. The inputs to the CRF, which are analogous to the emission probabilities in an HMM, will come from an LSTM.

But first, in order to test your decoder, you will also implement some functions to evaluate the output of an NER system according to two metrics:
1. You will count the number of _violations_ of the NER label constraints, i.e., how many times `I-TYPE` follows `O` or a tag of a different type or occurs at the beginning of a sentence. This number will be greater than 0 in the raw LSTM output, but should be 0 for your CRF output.
1. You will compute the _span-level_ precision, recall, and F1 of NER output. Although the baseline LSTM was trained to achieve high _token-level_ accuracy, this metric can be misleadingly high, since so many tokens are correctly labeled `O`. In other words, what proportion of spans predicted by the model line up exactly with spans in the gold standard, and what proportion of spans in the gold standard were predicted by the model? For more, see the original task definition: https://www.aclweb.org/anthology/W03-0419/.

We start with loading some code and data and the describe your tasks in more detail.

## Set Up Dependencies and Definitions

In [ ]:
!pip install --upgrade spacy allennlp
import spacy
print(spacy.__version__)

Requirement already up-to-date: spacy in /usr/local/lib/python3.6/dist-packages (2.2.4)
     |████████████████████████████████| 7.6MB 1.4MB/s 
     |████████████████████████████████| 133kB 43.0MB/s 
     |████████████████████████████████| 245kB 44.5MB/s 
     |████████████████████████████████| 5.5MB 36.9MB/s 
     |████████████████████████████████| 61kB 8.3MB/s 
     |████████████████████████████████| 256kB 45.4MB/s 
     |████████████████████████████████| 204kB 43.5MB/s 
     |████████████████████████████████| 51kB 7.5MB/s 
     |████████████████████████████████| 163kB 41.6MB/s 
     |████████████████████████████████| 51kB 6.9MB/s 
     |████████████████████████████████| 1.0MB 37.9MB/s 
  Created wheel for word2number: filename=word2number-1.1-cp36-none-any.whl size=5587 sha256=4aab8081f0dd706e61e08e5ff0e7e2a50bf76e0e2662e92239cf4d4a35b0dfb9
  Stored in directory: /root/.cache/pip/wheels/46/2f/53/5f5c1d275492f2fce1cdab9a9bb12d49286dead829a4078e0e
  Created wheel for ftfy: filename=ftf

In [ ]:
from typing import Iterator, List, Dict
import torch
import torch.optim as optim
import numpy as np
from allennlp.data import Instance
from allennlp.data.fields import TextField, SequenceLabelField
from allennlp.data.dataset_readers import DatasetReader
from allennlp.common.file_utils import cached_path
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer
from allennlp.data.tokenizers import Token
from allennlp.data.vocabulary import Vocabulary
from allennlp.models import Model
from allennlp.modules.text_field_embedders import TextFieldEmbedder, BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
from allennlp.modules.seq2seq_encoders import Seq2SeqEncoder, PytorchSeq2SeqWrapper
from allennlp.nn.util import get_text_field_mask, sequence_cross_entropy_with_logits
from allennlp.training.metrics import CategoricalAccuracy
from allennlp.data.iterators import BucketIterator
from allennlp.training.trainer import Trainer
from allennlp.predictors import SentenceTaggerPredictor
from allennlp.data.dataset_readers import conll2003

torch.manual_seed(1)

In [ ]:
class LstmTagger(Model):
  def __init__(self,
               word_embeddings: TextFieldEmbedder,
               encoder: Seq2SeqEncoder,
               vocab: Vocabulary) -> None:
    super().__init__(vocab)
    self.word_embeddings = word_embeddings
    self.encoder = encoder
    self.hidden2tag = torch.nn.Linear(in_features=encoder.get_output_dim(),
                                      out_features=vocab.get_vocab_size('labels'))
    self.accuracy = CategoricalAccuracy()

  def forward(self,
              tokens: Dict[str, torch.Tensor],
              metadata,
              tags: torch.Tensor = None) -> Dict[str, torch.Tensor]:
    mask = get_text_field_mask(tokens)
    embeddings = self.word_embeddings(tokens)
    encoder_out = self.encoder(embeddings, mask)
    tag_logits = self.hidden2tag(encoder_out)
    output = {"tag_logits": tag_logits}
    if tags is not None:
      self.accuracy(tag_logits, tags, mask)
      output["loss"] = sequence_cross_entropy_with_logits(tag_logits, tags, mask)

    return output

  def get_metrics(self, reset: bool = False) -> Dict[str, float]:
    return {"accuracy": self.accuracy.get_metric(reset)}

## Import Data

In [ ]:
reader = conll2003.Conll2003DatasetReader()
train_dataset = reader.read(cached_path('http://www.ccs.neu.edu/home/dasmith/onto.train.ner.sample'))
validation_dataset = reader.read(cached_path('http://www.ccs.neu.edu/home/dasmith/onto.development.ner.sample'))

vocab = Vocabulary.from_instances(train_dataset + validation_dataset)

159377B [00:00, 872113.13B/s]
562it [00:00, 15775.13it/s]
8366B [00:00, 6211638.74B/s]
23it [00:00, 3781.17it/s]
100%|██████████| 585/585 [00:00<00:00, 24410.23it/s]


## Define and Train Model

In [ ]:
EMBEDDING_DIM = 6
HIDDEN_DIM = 6
token_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),
                            embedding_dim=EMBEDDING_DIM)
word_embeddings = BasicTextFieldEmbedder({"tokens": token_embedding})
lstm = PytorchSeq2SeqWrapper(torch.nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, bidirectional=False, batch_first=True))
model = LstmTagger(word_embeddings, lstm, vocab)
if torch.cuda.is_available():
    cuda_device = 0
    model = model.cuda(cuda_device)
else:
    cuda_device = -1
# optimizer = optim.AdamW(model.parameters(), lr=1e-4, eps=1e-8)
optimizer = optim.SGD(model.parameters(), lr=0.1)
iterator = BucketIterator(batch_size=2, sorting_keys=[("tokens", "num_tokens")])
iterator.index_with(vocab)
trainer = Trainer(model=model,
                  optimizer=optimizer,
                  iterator=iterator,
                  train_dataset=train_dataset,
                  validation_dataset=validation_dataset,
                  patience=10,
                  num_epochs=100,
                  cuda_device=cuda_device)
trainer.train()

accuracy: 0.8442, loss: 0.9095 ||: 100%|██████████| 281/281 [00:02<00:00, 131.14it/s]
accuracy: 0.7878, loss: 1.2054 ||: 100%|██████████| 12/12 [00:00<00:00, 295.42it/s]
accuracy: 0.8442, loss: 0.7298 ||: 100%|██████████| 281/281 [00:01<00:00, 166.71it/s]
accuracy: 0.7878, loss: 1.1870 ||: 100%|██████████| 12/12 [00:00<00:00, 278.99it/s]
accuracy: 0.8442, loss: 0.7162 ||: 100%|██████████| 281/281 [00:01<00:00, 165.19it/s]
accuracy: 0.7878, loss: 1.1899 ||: 100%|██████████| 12/12 [00:00<00:00, 301.50it/s]
accuracy: 0.8442, loss: 0.7062 ||: 100%|██████████| 281/281 [00:01<00:00, 165.00it/s]
accuracy: 0.7878, loss: 1.1781 ||: 100%|██████████| 12/12 [00:00<00:00, 304.93it/s]
accuracy: 0.8442, loss: 0.6981 ||: 100%|██████████| 281/281 [00:01<00:00, 163.38it/s]
accuracy: 0.7878, loss: 1.1584 ||: 100%|██████████| 12/12 [00:00<00:00, 308.75it/s]
accuracy: 0.8442, loss: 0.6904 ||: 100%|██████████| 281/281 [00:01<00:00, 161.89it/s]
accuracy: 0.7878, loss: 1.1717 ||: 100%|██████████| 12/12 [00:00

{'best_epoch': 98,
 'best_validation_accuracy': 0.8795918367346939,
 'best_validation_loss': 0.37066657008836046,
 'epoch': 99,
 'peak_cpu_memory_MB': 1925.604,
 'peak_gpu_0_memory_MB': 326,
 'training_accuracy': 0.9262789958690817,
 'training_cpu_memory_MB': 1925.604,
 'training_duration': '0:02:59.436962',
 'training_epochs': 99,
 'training_gpu_0_memory_MB': 326,
 'training_loss': 0.19093986263323004,
 'training_start_epoch': 0,
 'validation_accuracy': 0.8755102040816326,
 'validation_loss': 0.38457751233363524}

## Evaluation

The simple code below creators a `predictor` object that applies the model to an input example and then loops over the examples in the validation set, printing out the input token, gold-standard output, and model output. You can see from these methods how to access data and model outputs for evaluation.

In [ ]:
predictor = SentenceTaggerPredictor(model, dataset_reader=reader)
printingtest = [predictor.predict_instance(i)['tag_logits'] for i in validation_dataset]
print(printingtest)

def tag_sentence(s):
  tag_ids = np.argmax(predictor.predict_instance(s)['tag_logits'], axis=-1)  
  fields = zip(s['tokens'], s['tags'], [model.vocab.get_token_from_index(i, 'labels') for i in tag_ids])
  return list(fields)

baseline_output = [tag_sentence(i) for i in validation_dataset]
print(baseline_output)

[[[10.952136993408203, -2.414402961730957, -1.0277020931243896, 3.9395673274993896, 1.9644677639007568, -4.290562152862549, 0.08193725347518921, -3.812394618988037, 2.9985413551330566, 1.6645221710205078, 1.4096730947494507, -0.949555516242981, 0.09710417687892914, 0.7021098732948303, 1.4609196186065674, 0.6612187623977661, 1.4599008560180664, 1.1869268417358398, -1.3185756206512451, 1.012312889099121, 0.038860827684402466, 0.24974986910820007, -0.8472910523414612, -0.5917830467224121, -1.4059494733810425, -0.2937629222869873, -0.5776299238204956, -1.1273901462554932, -1.122093915939331, -1.3319449424743652, -2.0777571201324463, -1.9905544519424438, -1.685223937034607, -1.9520235061645508], [7.864308834075928, -0.28347790241241455, -2.4958460330963135, 1.4477686882019043, 4.933307647705078, -3.231520414352417, 2.2813029289245605, -1.2428206205368042, 4.284031391143799, 2.7968673706054688, 3.0068979263305664, -0.9875996112823486, -0.5756310224533081, -1.2488023042678833, 0.1627463996410

Now, you can implement two evaluation functions: `violations` and `span_stats`.

In [ ]:
# TODO: count the number of NER label violations,
# such as O followed by I-TYPE or B-TYPE followed by
# I-OTHER_TYPE
# Take tagger output as input
def violations(tagged):
  count = 0
  for sentence in tagged:
    for i in range(1, len(sentence)):
      if 'I' == sentence[i][2][0]:
        if sentence[i-1][2][0]  == 'O':
          count = count + 1
        
        type = sentence[i][2][2:]
        prev_type = sentence[i-1][2][2:]
        prev_tag = sentence[i-1][2][0]

        if prev_tag == 'B' or prev_tag == 'I':
          if prev_type != type:
            count = count +1
  return count

# TODO: return the span-level precision, recall, and F1
# Take tagger output as input
def span_stats(tagged):
  tp  = 0
  tn = 0
  fp = 0
  fn = 0
  for sentence in tagged:
    for i in range(0, len(sentence)):
      if sentence[i][1] == 'O':
        if sentence[i][2] == 'O':
          tn = tn + 1
        else:
          fp = fp + 1
      else:
        if sentence[i][1] == sentence[i][2]:
          tp = tp + 1
        elif sentence[i][2] == 'O':
          fn = fn + 1
        elif sentence[i][2] != sentence[i][1]:
          fp = fp + 1
  precision = tp / (tp+fp)
  recall = tp / (tp+fn)
  f1 = 2*(recall * precision) / (recall + precision)
  return {'precision': precision,
          'recall': recall,
          'f1': f1}

## You can check how many violations are made by the model output in predictor.
violations(baseline_output)
span_stats(baseline_output)


{'f1': 0.5906040268456376,
 'precision': 0.5057471264367817,
 'recall': 0.7096774193548387}

## Decoding

Now you can finally implement the simple Viterbi decoder. The `predictor` object, when applied to an input sentence, first calculates the scores for each possible output tag for each token. See the line `predictor.predict_instance(i)['tag_logits']` in the code above.

Then, you will construct a transition matrix. You can use the code below to get a list of the tags the model knows about. For a set of K tags, construct a K-by-K matrix with a log(1)=0 when a transition between a given tag pair is valid and a log(0)=-infinity otherwise.

Finally, implement a Viterbi decoder that takes the predictor object and a dataset object and outputs tagged data, just like the `tag_sentence` function above. It should use the Viterbi algorithm with the (max, plus) semiring. You'll be working with sums of log probabilities instead of products of probabilties.

Run your `violations` function on the output of this decoder to make sure that there are no invalid tag transitions. Also, compare the span-level metrics on `baseline_output` and your new output using your `span_stats` function.

In [ ]:
def create_emission_shape(sentence):
  t  = list(sentence['tokens'])
  tokens_list = []
  for i in range(0,len(t)):
    tokens_list.append((t[i][0]))

  return np.asarray(tokens_list)


In [ ]:
import pandas as pd

def viterbi_decoder(sentence):
  
  labels = vocab.get_index_to_token_vocabulary('labels')
  
  #create and populate transmission probability matrix
  trans_p = pd.DataFrame(index = labels.values(), columns = labels.values()).fillna(0)
  
  labels = list(labels.values())  
  #create emission probability matrix
  emissions = pd.DataFrame(predictor.predict_instance(sentence)['tag_logits']) 
  emissions.index = create_emission_shape(sentence)  
  emissions.columns = labels

  


  for i in range(0,trans_p.shape[0]):
    for j in range(0,trans_p.shape[1]):  
      if labels[j] != 'O':
        prevType = labels[i][0]
        prevTag = labels[i][2:0]
        currType = labels[j][0]
        currTag = labels[j][2:]
        #check for violations and substitute -infinity in transition matrix
        if(currType == 'I' and prevType == 'O'):
          trans_p.iloc[i,j] = float('-inf')
        elif(currType == 'I' and prevType == 'I' and prevTag != currTag):
          trans_p.iloc[i,j] = float('-inf') 
        elif (currType == 'I' and prevType == 'B' and prevTag != currTag):
          trans_p.iloc[i,j] = float('-inf') 
        else:
          continue

   
    labels = vocab.get_index_to_token_vocabulary('labels')
    e_matrix = emissions.copy()
    e_matrix.iloc[i,j] = 0
    tags_matrix = emissions.copy()
    tags_matrix.iloc[i,j] = 0

    for i in range(0, e_matrix.shape[1]):
      if(e_matrix.columns[i][0] != 'I'):
        e_matrix.iloc[0,i] = emissions.iloc[0,i]
      else:
        e_matrix.iloc[0,i] = 0
    row = emissions.shape[0]
    col = emissions.shape[1]

    for i in range(1,row):
      for j in range(0,col):
        joinproblist = []
        jointprob = 0
        for k in range(0,col):          
          jointprob = emissions.iloc[i,j] + trans_p.iloc[k,j] + e_matrix.iloc[i-1,k]
          joinproblist.append(jointprob)
        maxValue = max(joinproblist)
        maxIndex   = joinproblist.index(maxValue)
        e_matrix.iloc[i,j ] = maxValue        
        tags_matrix.iloc[i,j] = maxIndex
    #walk backwards through trellis to find best tag sequence
    lastTag = list((e_matrix.iloc[-1:]).max())
    lastIndex  = lastTag.index(max(lastTag))
    
    output = []
    for i in range (emissions.shape[0]-1,0,-1):
      tag = labels[lastIndex]
      output.append(tag)
      lastIndex = tags_matrix.iloc[i,int(lastIndex)]
    tag = labels[lastIndex]
    output.append(tag)
    output.reverse()        
    return output
   









print(vocab.get_index_to_token_vocabulary('labels'))
viterbi_output = [viterbi_decoder(i) for i in validation_dataset]
print(viterbi_output)




{0: 'O', 1: 'B-GPE', 2: 'I-ORG', 3: 'I-DATE', 4: 'B-CARDINAL', 5: 'I-EVENT', 6: 'B-PERSON', 7: 'B-NORP', 8: 'B-DATE', 9: 'B-ORG', 10: 'B-LOC', 11: 'I-LOC', 12: 'I-FAC', 13: 'I-PERSON', 14: 'I-GPE', 15: 'I-CARDINAL', 16: 'B-EVENT', 17: 'I-TIME', 18: 'I-WORK_OF_ART', 19: 'B-ORDINAL', 20: 'B-FAC', 21: 'B-TIME', 22: 'I-LAW', 23: 'I-QUANTITY', 24: 'I-NORP', 25: 'I-MONEY', 26: 'B-MONEY', 27: 'B-WORK_OF_ART', 28: 'B-QUANTITY', 29: 'B-LAW', 30: 'B-PRODUCT', 31: 'I-PRODUCT', 32: 'B-PERCENT', 33: 'I-PERCENT'}


In [ ]:
count = 0
for sentence in viterbi_output:
  for i in range(1, len(sentence)):
    if 'I' == sentence[i][0][0]:
      if sentence[i-1][0][0]  == 'O':
        count = count + 1
      
      type = sentence[i][0][2:]
      prev_type = sentence[i-1][0][2:]
      prev_tag = sentence[i-1][0][0]

      if prev_tag == 'B' or prev_tag == 'I':
        if prev_type != type:
          count = count +1
print(count)

0
